In [1]:
import pandas as pd
import numpy as np

def load_raceline_start(path):
    df = pd.read_csv(path)
    
    # 보통 포맷: t,x,y,yaw,speed 또는 x,y,heading...
    # 너가 준 raceline 파일은 x,y 좌표가 있는 걸로 가정.
    # 가장 앞 row를 start pose로 사용.
    x0 = df.iloc[0]["x"]
    y0 = df.iloc[0]["y"]
    
    return float(x0), float(y0)

raceline_start = load_raceline_start("./raceline_playground.csv")
print("Raceline Start =", raceline_start)

def is_log_matching_raceline(log_path, raceline_start, pos_tol=1.5):
    """
    log_path : RunLogger CSV
    raceline_start: (x0, y0)
    pos_tol : 출발 위치 판단 허용 거리 (m)
    """
    try:
        df = pd.read_csv(log_path)
        # 첫 row
        x = df.iloc[0]["x"]
        y = df.iloc[0]["y"]
    except Exception as e:
        print("[WARN] CSV 파싱 실패:", log_path)
        return False
    
    dx = float(x) - raceline_start[0]
    dy = float(y) - raceline_start[1]
    dist = np.hypot(dx, dy)
    
    return dist <= pos_tol


Raceline Start = (-10.0, 7.475)


In [2]:
import os
import re
import io
import math
import numpy as np
import pandas as pd

log_dir = "./logs/fgm"   # 필요하면 여기만 바꿔

# 파일명 패턴: Opp_bumper..._Gap..._Bub..._SPD..._FGM_..._dur_...s.csv
log_pattern = re.compile(
    r"^Opp_.*"                     # Opp_로 시작
    r"_FFG[-\d.]+"                 # _FFG<fgm_fov_angle>
    r"_SCD[-\d.]+"                 # _SCD<fgm_speed_check_fov_deg>
    r"_Gap[-\d.]+"                 # _Gap<...>
    r"_Bub[-\d.]+"                 # _Bub<...>
    r"_Clr[-\d.]+"                 # _Clr<...>
    r"_WW[-\d.]+"                  # _WW<...>
    r"_AW[-\d.]+"                  # _AW<...>
    r"_SW[-\d.]+"                  # _SW<...>
    r"_HB[-\d.]+"                  # _HB<...>
    r"_CT[-\d.]+"                  # _CT<...>
    r"_SA[-\d.]+"                  # _SA<...>
    r"_DBS[-\d.]+"                 # _DBS<...>
    r"_SPD[-\d.]+"                 # _SPD<...>
    r"_FGM_.*_dur_[-\d.]+s\.csv$"  # _FGM_<...>_dur_<t>s.csv
)

all_files = [
    os.path.join(log_dir, f)
    for f in os.listdir(log_dir)
    if f.endswith(".csv") and log_pattern.match(f)
]

print(f"발견된 로그 파일 개수: {len(all_files)}")
for f in all_files[:5]:
    print(" -", os.path.basename(f))

filtered_files = []
for f in all_files:
    if is_log_matching_raceline(f, raceline_start, pos_tol=1.5):
        filtered_files.append(f)
    else:
        print("❌ [FILTERED] Start position mismatch:", f)

print(f"✓ Valid logs: {len(filtered_files)} / {len(all_files)}")


발견된 로그 파일 개수: 32
 - Opp_bumper_slow_1.5.csv_FFG183.9030_SCD26.1100_Gap0.9520_Bub0.4680_Clr0.5980_WW0.6020_AW5.7420_SW0.0820_HB2.2450_CT0.1880_SA0.5010_DBS0.1010_SPD3.9200_FGM_1764875462_GOAL_dur_16.93s.csv
 - Opp_bumper_slow_1.csv_FFG183.9030_SCD26.1100_Gap0.9520_Bub0.4680_Clr0.5980_WW0.6020_AW5.7420_SW0.0820_HB2.2450_CT0.1880_SA0.5010_DBS0.1010_SPD3.9200_FGM_1764875385_GOAL_dur_12.06s.csv
 - Opp_bumper_v_5.csv_FFG169.8840_SCD27.8500_Gap0.9890_Bub0.4950_Clr0.5600_WW0.5850_AW6.0500_SW0.0880_HB2.0400_CT0.2600_SA0.4950_DBS0.1090_SPD3.7600_FGM_1764875972_GOAL_dur_11.06s.csv
 - Opp_bumper_slow_1.csv_FFG169.8840_SCD27.8500_Gap0.9890_Bub0.4950_Clr0.5600_WW0.5850_AW6.0500_SW0.0880_HB2.0400_CT0.2600_SA0.4950_DBS0.1090_SPD3.7600_FGM_1764875430_GOAL_dur_0.00s.csv
 - Opp_bumper_slow_1.csv_FFG178.4420_SCD28.3300_Gap1.0420_Bub0.5120_Clr0.5730_WW0.6580_AW6.2150_SW0.0705_HB1.9820_CT0.2140_SA0.5530_DBS0.1420_SPD3.4800_FGM_1764875321_ABORT_dur_0.00s.csv
❌ [FILTERED] Start position mismatch: ./logs/fgm\O

In [3]:
def load_run_log(path: str):
    """
    하나의 run_logger csv 파일을
    - df_data: per-step 데이터 (t,x,y,...)
    - summary_dict: 맨 아래 요약 블록 (key,value)
    로 나눠서 반환.
    """
    with open(path, "r") as f:
        lines = f.readlines()
    
    # Summary 시작 라인 찾기
    summary_idx = None
    for i, line in enumerate(lines):
        if line.strip().startswith('--- Experiment Summary'):
            summary_idx = i
            break
    
    if summary_idx is None:
        # Summary 없는 경우 (비상용)
        data_text = "".join(lines)
        df = pd.read_csv(io.StringIO(data_text))
        summary = {}
    else:
        # 맨 위 ~ summary 직전 빈 줄까지만 데이터로 사용
        data_text = "".join(lines[:summary_idx-1])
        df = pd.read_csv(io.StringIO(data_text))
        
        # summary 부분은 "key,value" 형식
        summary_lines = [l.strip() for l in lines[summary_idx+1:] if l.strip()]
        summary = {}
        for l in summary_lines:
            if "," in l:
                k, v = l.split(",", 1)
                summary[k.strip()] = v.strip()
    
    return df, summary


In [4]:
def parse_params_from_filename(path: str):
    """
    파일 이름에서
      scenario, Gap, Bub, Clr, WW, AW, SW, HB, CT, SA, DBS, SPD, result, duration_from_name
    를 뽑아서 dict로 반환.
    """
    name = os.path.basename(path)
    
    # 파라미터 블럭 파싱
    param_pattern = re.compile(
        r"_FFG(?P<FFG>[-\d.]+)"
        r"_SCD(?P<SCD>[-\d.]+)"
        r"_Gap(?P<Gap>[-\d.]+)"
        r"_Bub(?P<Bub>[-\d.]+)"
        r"_Clr(?P<Clr>[-\d.]+)"
        r"_WW(?P<WW>[-\d.]+)"
        r"_AW(?P<AW>[-\d.]+)"
        r"_SW(?P<SW>[-\d.]+)"
        r"_HB(?P<HB>[-\d.]+)"
        r"_CT(?P<CT>[-\d.]+)"
        r"_SA(?P<SA>[-\d.]+)"
        r"_DBS(?P<DBS>[-\d.]+)"
        r"_SPD(?P<SPD>[-\d.]+)"
    )

    
    m = param_pattern.search(name)
    params = {}
    if m:
        for k, v in m.groupdict().items():
            try:
                params[k] = float(v)
            except ValueError:
                params[k] = None
    
    # scenario (Gap 앞부분 전체)
    scen = name.split("_Gap")[0]
    params["scenario"] = scen   # ex) Opp_bumper_slow_0.5.csv
    
    # 결과 태그 (GOAL / CRASH / STUCK / ABORT 등)
    m2 = re.search(r"_(GOAL|CRASH|STUCK|ABORT)_", name)
    if m2:
        params["result"] = m2.group(1)
    
    # 파일명에 있는 duration
    m3 = re.search(r"_dur_([0-9.]+)s\.csv$", name)
    if m3:
        try:
            params["duration_from_name"] = float(m3.group(1))
        except ValueError:
            params["duration_from_name"] = None
    
    return params


In [5]:
def parse_summary_types(summary: dict):
    """
    summary dict 의 value를 적당히 숫자/불리언으로 캐스팅.
    """
    out = {}
    for k, v in summary.items():
        v = v.strip()
        # 숫자 시도
        try:
            out[k] = float(v)
            continue
        except ValueError:
            pass
        
        # YES/NO -> 1/0
        if v.upper() in ("YES", "NO"):
            out[k] = 1 if v.upper() == "YES" else 0
        else:
            out[k] = v
    return out


In [6]:
def compute_metrics(df: pd.DataFrame, safe_dist: float = 0.5):
    """
    run_logger 데이터 df 에서
    - total_time
    - collision_flag
    - min_dist / unsafe_ratio
    - fgm_min_dist / fgm_unsafe_ratio / fgm_track_rms
    를 계산 (있으면).
    """
    metrics = {}
    
    # 총 시간
    if "t" in df.columns and len(df) > 1:
        metrics["total_time"] = float(df["t"].iloc[-1] - df["t"].iloc[0])
    else:
        metrics["total_time"] = 0.0
    
    # 충돌 플래그
    if "collision" in df.columns:
        metrics["collision_flag"] = int(df["collision"].max() > 0)
    else:
        metrics["collision_flag"] = 0
    
    # 전역 min_d / unsafe_ratio
    if "min_d" in df.columns:
        metrics["min_dist"] = float(df["min_d"].min())
        metrics["unsafe_ratio"] = float((df["min_d"] < safe_dist).mean())
    
    # FGM 전용
    if "planner" in df.columns and "fgm_min_d" in df.columns:
        mask_fgm = df["planner"].astype(str).str.contains("FGM")
        if mask_fgm.any():
            df_fgm = df[mask_fgm]
            metrics["fgm_min_dist"] = float(df_fgm["fgm_min_d"].min())
            metrics["fgm_unsafe_ratio"] = float((df_fgm["fgm_min_d"] < safe_dist).mean())
            if "fgm_track" in df_fgm.columns:
                metrics["fgm_track_rms"] = float(math.sqrt((df_fgm["fgm_track"] ** 2).mean()))
    
    return metrics


In [7]:
def analyze_log(path: str, safe_dist: float = 0.5):
    """
    하나의 로그 파일 경로를 받아서
    - 파일 이름 기반 파라미터
    - summary 블럭 내용
    - per-step metrics
    다 합쳐서 dict 로 리턴.
    """
    df, summary = load_run_log(path)
    params   = parse_params_from_filename(path)
    metrics  = compute_metrics(df, safe_dist=safe_dist)
    summary_parsed = parse_summary_types(summary)
    
    row = {}
    row.update(params)
    row.update(summary_parsed)
    row.update(metrics)
    row["log_path"] = path
    
    return row


In [8]:
rows = []
for p in filtered_files:
    row = analyze_log(p, safe_dist=0.5)
    rows.append(row)

summary_df = pd.DataFrame(rows)
print("정리된 행 개수:", len(summary_df))
summary_df.head()


정리된 행 개수: 27


,FFG,SCD,Gap,Bub,Clr,WW,AW,SW,HB,CT,...,Min Dist (m),Unsafe Ratio,FGM Min Dist (m),FGM Unsafe Ratio,FGM Track RMS (m),total_time,collision_flag,min_dist,unsafe_ratio,log_path
0,183.903,26.11,0.952,0.468,0.598,0.602,5.742,0.082,2.245,0.188,...,0.0,1.0,1.000000e+09,0.0,0.0,16.925269,0,0.0,1.0,./logs/fgm\Opp_bumper_slow_1.5.csv_FFG183.9030...
1,183.903,26.11,0.952,0.468,0.598,0.602,5.742,0.082,2.245,0.188,...,0.0,1.0,1.000000e+09,0.0,0.0,12.057642,0,0.0,1.0,./logs/fgm\Opp_bumper_slow_1.csv_FFG183.9030_S...
2,169.884,27.85,0.989,0.495,0.560,0.585,6.050,0.088,2.040,0.260,...,0.0,1.0,1.000000e+09,0.0,0.0,11.059226,0,0.0,1.0,./logs/fgm\Opp_bumper_v_5.csv_FFG169.8840_SCD2...
3,169.884,27.85,0.989,0.495,0.560,0.585,6.050,0.088,2.040,0.260,...,0.0,1.0,1.000000e+09,0.0,0.0,11.065628,0,0.0,1.0,./logs/fgm\Opp_bumper_v_3.csv_FFG169.8840_SCD2...
4,183.903,26.11,0.952,0.468,0.598,0.602,5.742,0.082,2.245,0.188,...,0.0,1.0,1.000000e+09,0.0,0.0,10.637468,0,0.0,1.0,./logs/fgm\Opp_bumper_v_3.csv_FFG183.9030_SCD2...


In [9]:
def get_first_existing(row, candidates, default=np.nan):
    for c in candidates:
        if c in row and not pd.isna(row[c]):
            return row[c]
    return default


In [10]:
def compute_safety_comfort_score(row,
                                 d_safe=0.5,
                                 d_good=0.8,
                                 track_rms_bad=1.0,
                                 w_safety=0.6,
                                 w_progress=0.3,
                                 w_comfort=0.1):
    """
    progress는 아직 모르는 상태라 safety+comfort만 보고 임시 점수 계산.
    (progress는 나중에 duration min/max로 정규화해서 반영)
    """
    # 충돌 / 비정상 종료 체크
    result = str(row.get("result", "")).upper()
    collision_flag = 0
    if "Collision" in row:
        collision_flag = int(row["Collision"])
    elif "collision_flag" in row:
        collision_flag = int(row["collision_flag"])
    
    if collision_flag == 1 or result in ("CRASH", "STUCK", "ABORT"):
        return 0.0
    
    # ----- Safety -----
    fgm_min = get_first_existing(row, ["FGM Min Dist (m)", "fgm_min_dist", "Min Dist (m)", "min_dist"], default=np.nan)
    if math.isnan(fgm_min):
        fgm_min = d_safe
    
    fgm_unsafe = get_first_existing(row, ["FGM Unsafe Ratio", "fgm_unsafe_ratio", "Unsafe Ratio", "unsafe_ratio"], default=0.0)
    if math.isnan(fgm_unsafe):
        fgm_unsafe = 0.0
    
    # 최소 거리 점수
    Sd = (fgm_min - d_safe) / (d_good - d_safe) if d_good > d_safe else 1.0
    Sd = max(0.0, min(1.0, Sd))
    
    # Unsafe Ratio 점수
    Su = 1.0 - max(0.0, min(1.0, fgm_unsafe))
    
    S_safety = 0.7 * Sd + 0.3 * Su
    
    # ----- Comfort (트랙 RMS) -----
    track_rms = get_first_existing(row, ["FGM Track RMS (m)", "fgm_track_rms"], default=0.0)
    if math.isnan(track_rms):
        track_rms = 0.0
    
    S_track = 1.0 - max(0.0, min(1.0, track_rms / track_rms_bad))
    S_comfort = S_track
    
    # progress 제외 상태에서 임시 점수 (safety+comfort 비율만 유지)
    # w_progress는 일단 무시하고, (w_safety + w_comfort)만 사용
    w_sc = w_safety + w_comfort
    Score0 = 0.0
    if w_sc > 0:
        Score0 = (w_safety * S_safety + w_comfort * S_comfort) / w_sc
    return Score0 * 100.0


In [11]:
# 1) safety/comfort 기반 임시 점수 계산
summary_df["Score_tmp"] = summary_df.apply(compute_safety_comfort_score, axis=1)

# 2) duration_raw 칼럼 준비 (Summary > 파일명 순으로)
summary_df["duration_raw"] = summary_df.apply(
    lambda r: get_first_existing(r, ["Duration (s)", "duration_from_name"], default=np.nan),
    axis=1
)

# 3) duration min/max (충돌/ABORT 제외)
valid_mask = (summary_df["Score_tmp"] > 0) & (~summary_df["duration_raw"].isna())
if valid_mask.any():
    T_min = summary_df.loc[valid_mask, "duration_raw"].min()
    T_max = summary_df.loc[valid_mask, "duration_raw"].max()
else:
    T_min, T_max = np.nan, np.nan

def compute_progress(duration, T_min, T_max):
    if math.isnan(duration) or math.isnan(T_min) or math.isnan(T_max) or T_max <= T_min:
        return 0.0
    val = (T_max - duration) / (T_max - T_min)
    return max(0.0, min(1.0, val))

summary_df["S_progress"] = summary_df["duration_raw"].apply(
    lambda t: compute_progress(t, T_min, T_max)
)

def compute_final_score(row,
                        w_safety=0.6,
                        w_progress=0.3,
                        w_comfort=0.1):
    # 충돌 / 비정상 종료면 0점
    result = str(row.get("result", "")).upper()
    collision_flag = 0
    if "Collision" in row:
        collision_flag = int(row["Collision"])
    elif "collision_flag" in row:
        collision_flag = int(row["collision_flag"])
    if collision_flag == 1 or result in ("CRASH", "STUCK", "ABORT"):
        return 0.0
    
    # safety+comfort는 Score_tmp 에 이미 들어 있음 (0~100)
    Score_tmp = row["Score_tmp"]
    # progress는 0~1
    S_progress = row["S_progress"]
    
    # 안전+컴포트 점수 비율 = (1 - w_progress)
    # 최종: Score = (1 - w_progress)*Score_tmp + w_progress*(100 * S_progress)
    Score_final = (1.0 - w_progress) * Score_tmp + w_progress * (100.0 * S_progress)
    
    # 클립
    return max(0.0, min(100.0, Score_final))

summary_df["Score"] = summary_df.apply(compute_final_score, axis=1)

# 점수 높은 순으로 보기
summary_df_sorted = summary_df.sort_values("Score", ascending=False)
summary_df_sorted.head(10)[[
    "scenario", "result",
    "FFG", "SCD", "Gap", "Bub", "Clr", "WW", "AW", "SW", "HB", "CT", "SA", "DBS", "SPD",
    "Score",
    "FGM Min Dist (m)","FGM Unsafe Ratio","FGM Track RMS (m)",
    "Duration (s)","duration_from_name"
]]


,scenario,result,FFG,SCD,Gap,Bub,Clr,WW,AW,SW,...,CT,SA,DBS,SPD,Score,FGM Min Dist (m),FGM Unsafe Ratio,FGM Track RMS (m),Duration (s),duration_from_name
23,Opp_bumper_v_5.csv_FFG183.9030_SCD26.1100,GOAL,183.903,26.11,0.952,0.468,0.598,0.602,5.742,0.082,...,0.188,0.501,0.101,3.92,100.000000,1.000000e+09,0.0,0.0,10.53,10.53
22,Opp_bumper_v_4.csv_FFG183.9030_SCD26.1100,GOAL,183.903,26.11,0.952,0.468,0.598,0.602,5.742,0.082,...,0.188,0.501,0.101,3.92,99.560117,1.000000e+09,0.0,0.0,10.63,10.63
4,Opp_bumper_v_3.csv_FFG183.9030_SCD26.1100,GOAL,183.903,26.11,0.952,0.468,0.598,0.602,5.742,0.082,...,0.188,0.501,0.101,3.92,99.516129,1.000000e+09,0.0,0.0,10.64,10.64
17,Opp_bumper_v_2.csv_FFG183.9030_SCD26.1100,GOAL,183.903,26.11,0.952,0.468,0.598,0.602,5.742,0.082,...,0.188,0.501,0.101,3.92,99.384164,1.000000e+09,0.0,0.0,10.67,10.67
12,Opp_bumper_v_5.csv_FFG174.1200_SCD25.4500,GOAL,174.120,25.45,1.128,0.537,0.542,0.745,6.890,0.076,...,0.232,0.472,0.118,3.30,98.856305,1.000000e+09,0.0,0.0,10.79,10.79
24,Opp_bumper_v_4.csv_FFG174.1200_SCD25.4500,GOAL,174.120,25.45,1.128,0.537,0.542,0.745,6.890,0.076,...,0.232,0.472,0.118,3.30,98.680352,1.000000e+09,0.0,0.0,10.83,10.83
25,Opp_bumper_v_4.csv_FFG169.8840_SCD27.8500,GOAL,169.884,27.85,0.989,0.495,0.560,0.585,6.050,0.088,...,0.260,0.495,0.109,3.76,97.844575,1.000000e+09,0.0,0.0,11.02,11.02
2,Opp_bumper_v_5.csv_FFG169.8840_SCD27.8500,GOAL,169.884,27.85,0.989,0.495,0.560,0.585,6.050,0.088,...,0.260,0.495,0.109,3.76,97.668622,1.000000e+09,0.0,0.0,11.06,11.06
3,Opp_bumper_v_3.csv_FFG169.8840_SCD27.8500,GOAL,169.884,27.85,0.989,0.495,0.560,0.585,6.050,0.088,...,0.260,0.495,0.109,3.76,97.624633,1.000000e+09,0.0,0.0,11.07,11.07
18,Opp_bumper_v_2.csv_FFG174.1200_SCD25.4500,GOAL,174.120,25.45,1.128,0.537,0.542,0.745,6.890,0.076,...,0.232,0.472,0.118,3.30,97.272727,1.000000e+09,0.0,0.0,11.15,11.15


In [12]:
param_cols = [    "FFG", "SCD", "Gap", "Bub", "Clr", "WW", "AW", "SW", "HB", "CT", "SA", "DBS", "SPD"]

grouped = (
    summary_df
    .groupby(param_cols)
    .agg(
        mean_score=("Score", "mean"),
        min_score=("Score", "min"),
        max_score=("Score", "max"),
        runs=("Score", "count"),
        crashes=("Collision", "sum") if "Collision" in summary_df.columns else ("collision_flag", "sum")
    )
    .reset_index()
    .sort_values("mean_score", ascending=False)
)

grouped


,FFG,SCD,Gap,Bub,Clr,WW,AW,SW,HB,CT,SA,DBS,SPD,mean_score,min_score,max_score,runs,crashes
2,178.442,28.33,1.042,0.512,0.573,0.658,6.215,0.0705,1.982,0.214,0.553,0.142,3.48,94.684751,90.234604,96.876833,6,0
3,183.903,26.11,0.952,0.468,0.598,0.602,5.742,0.0820,2.245,0.188,0.501,0.101,3.92,93.792155,71.847507,100.000000,8,0
1,174.120,25.45,1.128,0.537,0.542,0.745,6.890,0.0760,1.865,0.232,0.472,0.118,3.30,92.785924,84.120235,98.856305,6,0
0,169.884,27.85,0.989,0.495,0.560,0.585,6.050,0.0880,2.040,0.260,0.495,0.109,3.76,91.686217,70.000000,97.844575,7,0


In [13]:
import numpy as np
import pandas as pd

# FFG = fgm_fov_angle
# SCD = fgm_speed_check_fov_deg
param_cols = [
    "FFG", "SCD",   # 새로 추가된 두 파라미터
    "Gap", "Bub", "Clr", "WW", "AW", "SW", "HB", "CT", "SA", "DBS", "SPD"
]

# 각 파라미터 튜닝 범위 (대략 이전에 쓰던 값 기준으로 적당히 설정)
param_bounds = {
    "FFG": (140.0, 210.0),   # fov angle [deg]
    "SCD": (10.0,  45.0),    # speed check fov [deg]

    "Gap": (0.8,  2.0),
    "Bub": (0.3,  0.8),
    "Clr": (0.4,  0.8),
    "WW":  (0.2,  1.0),
    "AW":  (2.0, 10.0),
    "SW":  (0.02, 0.2),
    "HB":  (0.0,  4.0),
    "CT":  (0.15, 0.5),
    "SA":  (0.2,  0.8),
    "DBS": (0.05, 0.3),
    "SPD": (2.5,  6.0),
}


In [14]:
rng = np.random.default_rng(42)  # 필요하면 42 말고 다른 숫자로 바꿔도 됨

def propose_new_params_simple(
    grouped: pd.DataFrame,
    param_cols,
    param_bounds,
    n_seeds=3,       # mean_score 상위 몇 세트를 seed로 쓸지
    per_seed=10,     # seed 하나당 몇 개 후보를 생성할지
    noise_scale=0.08 # ±8% 정도 multiplicative noise
):
    """
    grouped: FFG,SCD,Gap~SPD, mean_score 등이 들어있는 DataFrame
    param_cols: 튜닝할 파라미터 컬럼 이름 리스트
    param_bounds: 각 파라미터 별 (min,max) 딕셔너리
    """
    # 1) mean_score 높은 순서대로 상위 n_seeds 뽑기
    top = grouped.sort_values("mean_score", ascending=False).head(n_seeds)
    
    new_rows = []
    for _, seed in top.iterrows():
        base = seed[param_cols].astype(float).values  # seed 파라미터 벡터
        
        for _ in range(per_seed):
            # multiplicative gaussian noise: θ' = θ * (1 + N(0, noise_scale))
            noise = rng.normal(loc=0.0, scale=noise_scale, size=len(param_cols))
            cand = base * (1.0 + noise)
            
            # 각 파라미터 별 bounds 안으로 자르기
            cand_clipped = []
            for val, name in zip(cand, param_cols):
                lo, hi = param_bounds[name]
                cand_clipped.append(float(np.clip(val, lo, hi)))
            
            new_rows.append(dict(zip(param_cols, cand_clipped)))
    
    new_df = pd.DataFrame(new_rows)
    
    # 2) 이미 실험한 파라미터 세트와 완전히 같은 조합은 제거 (중복 방지)
    merged = new_df.merge(grouped[param_cols], on=param_cols, how="left", indicator=True)
    new_unique = merged[merged["_merge"] == "left_only"][param_cols].reset_index(drop=True)
    
    return new_unique


In [15]:
new_candidates = propose_new_params_simple(
    grouped,
    param_cols=param_cols,
    param_bounds=param_bounds,
    n_seeds=3,      # mean_score 상위 3개를 seed로
    per_seed=10,    # seed 하나당 10개씩 → 총 30개 후보
    noise_scale=0.08
)

print("새로 제안된 파라미터 개수:", len(new_candidates))
new_candidates.head()


새로 제안된 파라미터 개수: 30


,FFG,SCD,Gap,Bub,Clr,WW,AW,SW,HB,CT,SA,DBS,SPD
0,182.791946,25.972980,1.104558,0.550526,0.483565,0.589453,6.278562,0.068716,1.979336,0.199396,0.591905,0.150836,3.498383
1,194.533774,29.389563,0.970369,0.527104,0.529045,0.704242,6.190177,0.069457,1.874032,0.234930,0.546164,0.137134,3.381966
2,186.040905,29.158242,1.076405,0.529646,0.671173,0.636606,5.960313,0.065910,2.079670,0.233328,0.547959,0.132456,3.250464
3,187.729446,30.014511,1.087277,0.484741,0.583642,0.664142,6.323732,0.075415,2.017453,0.225623,0.555990,0.145284,3.655751
4,157.640576,27.605497,1.002790,0.485832,0.560387,0.736694,5.784509,0.075961,1.715164,0.208267,0.560200,0.148659,3.678005


In [16]:
def df_to_params_list_strings(df):
    """
    new_candidates DataFrame(FFG,SCD,Gap~SPD 13개) → bash params_list용 문자열 리스트
    """
    lines = []
    for _, row in df.iterrows():
        vals = [
            row["Gap"],
            row["Bub"],
            row["FFG"],
            row["SCD"],
            row["Clr"],
            row["WW"],
            row["AW"],
            row["SW"],
            row["HB"],
            row["CT"],
            row["SA"],
            row["DBS"],
            row["SPD"],
            
        ]
        s = " ".join(f"{v:.4f}" for v in vals)
        lines.append(f"\"{s}\"")
    return lines

param_strings = df_to_params_list_strings(new_candidates)

print("params_list=(")
for s in param_strings:
    print("    " + s)
print(")")


params_list=(
    "1.1046 0.5505 182.7919 25.9730 0.4836 0.5895 6.2786 0.0687 1.9793 0.1994 0.5919 0.1508 3.4984"
    "0.9704 0.5271 194.5338 29.3896 0.5290 0.7042 6.1902 0.0695 1.8740 0.2349 0.5462 0.1371 3.3820"
    "1.0764 0.5296 186.0409 29.1582 0.6712 0.6366 5.9603 0.0659 2.0797 0.2333 0.5480 0.1325 3.2505"
    "1.0873 0.4847 187.7294 30.0145 0.5836 0.6641 6.3237 0.0754 2.0175 0.2256 0.5560 0.1453 3.6558"
    "1.0028 0.4858 157.6406 27.6055 0.5604 0.7367 5.7845 0.0760 1.7152 0.2083 0.5602 0.1487 3.6780"
    "1.0035 0.5471 189.7673 27.5396 0.5642 0.5908 5.6515 0.0653 2.0608 0.2164 0.5835 0.1371 3.5241"
    "1.0801 0.4849 187.3725 27.6289 0.5564 0.6379 5.6204 0.0732 1.9076 0.2142 0.5743 0.1471 3.6652"
    "1.0354 0.4429 177.0361 27.3706 0.5067 0.5884 5.7192 0.0728 1.8384 0.2075 0.6105 0.1380 3.6853"
    "0.9628 0.4981 165.1143 27.8644 0.6115 0.5671 6.4310 0.0718 1.8878 0.1892 0.5562 0.1360 3.5448"
    "1.0220 0.4701 178.7539 31.9603 0.5812 0.6696 6.8908 0.0752 2.0386 0.2391 0.5004 0